In [2]:
import os
import torch
import transformers
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, AutoTokenizer
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from whisper_mic import WhisperMic
import warnings
warnings.filterwarnings('ignore')

# Set device and torch dtype for the model on the GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [3]:
from transformers import pipeline
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [6]:



try:
    mic = WhisperMic()
    result = mic.listen()
    print(result)
    promptinput = result
except Exception as e:
    print(f"An error occurred: {e}")


[02/20/24 21:17:38] INFO     No mic index provided, using default                                 ]8;id=285394;file://c:\Users\harsh\anaconda3\envs\development\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=614763;file://c:\Users\harsh\anaconda3\envs\development\lib\site-packages\whisper_mic\whisper_mic.py#84\84]8;;\

[02/20/24 21:17:40] INFO     Mic setup complete                                                   ]8;id=533329;file://c:\Users\harsh\anaconda3\envs\development\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=742851;file://c:\Users\harsh\anaconda3\envs\development\lib\site-packages\whisper_mic\whisper_mic.py#95\95]8;;\

                    INFO     Listening...                                                        ]8;id=343058;file://c:\Users\harsh\anaconda3\envs\development\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=114617;file://c:\Users\harsh\anaconda3\envs\development\lib\site-packages\whisper_mic\whisper_mic.py#214\214]8;;\

 What is the capital of India?


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY2"))


question = promptinput

# Call the OpenAI API to get a response to the user's question
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "assistant", "content": "You are a helpful assistant designed to answer user queries."},
        {"role": "user", "content": question}
    ]
)

# Extract the generated text from the response
print(completion.choices[0].message)


In [45]:
# as we are able to convert the speech to text, we can now use the text to text model


t2tmodel="meta-llama/Llama-2-7b-chat-hf"
tokenizer=AutoTokenizer.from_pretrained(t2tmodel)
 # create Huggingface pipeline

pipeline = transformers.pipeline("text-generation",
     model=t2tmodel,
     tokenizer=tokenizer,
     torch_dtype=torch.bfloat16,
     trust_remote_code=True,
     device_map="auto",
     max_length=1000,
     do_sample=True,
     top_k=10,
     num_return_sequences=1,
     eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
print(t2tmodel)

meta-llama/Llama-2-7b-chat-hf


In [26]:
print(pipeline)

In [27]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [28]:
t2tresult=llm.invoke(promptinput)

In [34]:
zero_shot_prompt = "Write a creative story about..."

In [ ]:
t2tresult = llm.invoke(prompt=zero_shot_prompt,input=promptinput, model_kwargs={'temperature':0.7})

In [40]:
print(t2tresult)



A. Mumbai
B. Delhi
C. Kolkata
D. Chennai

Answer: B. Delhi

Explanation: Delhi is the capital city of India.


In [37]:
# converting text to audio
import playsound
from gtts import gTTS
tts = gTTS(text=t2tresult, lang='en')  # Replace 'en' with your desired language
tts.save('output.mp3')




In [ ]:
# Play the audio file
playsound("generated_audio.mp3")